Jack Forman <br>
CEM Lab <br>
Data Sorter <br>

In [395]:
import pandas as pd
import os
import math
import matplotlib.pyplot as plt
import numpy as np


User Data

In [396]:
Tracksdatalabels = ['LINEARITY_OF_FORWARD_PROGRESSION', 'TRACK_MEAN_SPEED', 'TRACK_DISPLACEMENT'] # Name of measurements in data from #####
Spotsdatalabels = []

outputloc = r"E:\CEMLab\MigrationAssays\SingleCellAnalysis\AllData.xlsx" # where data is exported to

firstTracks = r"E:\CEMLab\MigrationAssays\SingleCellAnalysis\5192023\C1_RTracks.csv" 
firstSpots = r"E:\CEMLab\MigrationAssays\SingleCellAnalysis\5172023\C1_LSpots.csv"

groups = ['C', 'G', 'T']
folders = [
    r"E:\CEMLab\MigrationAssays\SingleCellAnalysis\5172023",
    r"E:\CEMLab\MigrationAssays\SingleCellAnalysis\5192023"
]

Tracks sorting function

In [397]:
def isfloat(value):
    try: 
        float(value)
        return True
    except ValueError:
        return False
def nancheck(value):
    try:
        math.isnan(value)
        return False
    except TypeError:
        return True



def maindatasortTracks(loc, datalabels):
    df = pd.read_csv(loc)
    #print(df)
    outputdic = {}
    for l in datalabels:
        outputdic[l] = [float(x) for x in df[l] if isfloat(x)]
    return outputdic





Spots Sorting Function

In [398]:
def spotssorting(loc):
    '''
    Takes in a location
    exports data 
    
    '''
    data = {}
    df = pd.read_csv(loc)
    initial = df[df['FRAME']== '0']
    x = [float(x) for x in initial['POSITION_X'] if isfloat(x)]
    xbar = np.mean(x)
    if False == True:
        plt.hist(x)
        plt.show()
    trackids = [x for x in df['TRACK_ID'] if isfloat(x) and nancheck(x)]
    trackids = set(trackids)
    #trackids = [float(x) for x in df['TRACK_ID'] if isfloat(x)]
    #trackids = set(trackids)
    for id in trackids:
        data[id] = {}
        dfid = df[df['TRACK_ID'] == id]
        frames = [int(x) for x in dfid['FRAME'] if isfloat(x)]
        frames = set(frames)
        minframe = min(frames)
        frames = [str(x) for x in frames]
        xi = float(dfid[dfid['FRAME'] == str(minframe)]['POSITION_X'].iloc[0])
        
        for f in frames:
            if xi < xbar:
                if str(int(f) + 1) in frames:
                    dx = float(dfid[dfid['FRAME'] == str(int(f) + 1)]['POSITION_X'].iloc[0]) - float(dfid[dfid['FRAME'] == f]['POSITION_X'].iloc[0])
                    dy = float(dfid[dfid['FRAME'] == str(int(f) + 1)]['POSITION_Y'].iloc[0]) - float(dfid[dfid['FRAME'] == f]['POSITION_Y'].iloc[0])
                    data[id][f] = abs(math.degrees(math.atan2(dy, dx)))
            elif xi > xbar:
                if str(int(f) + 1) in frames:
                    dx = float(dfid[dfid['FRAME'] == str(int(f) + 1)]['POSITION_X'].iloc[0]) - float(dfid[dfid['FRAME'] == f]['POSITION_X'].iloc[0])
                    dy = float(dfid[dfid['FRAME'] == str(int(f) + 1)]['POSITION_Y'].iloc[0]) - float(dfid[dfid['FRAME'] == f]['POSITION_Y'].iloc[0])
                    data[id][f] = 180 - abs(math.degrees(math.atan2(dy, dx)))
    return data


Batch Processing

In [399]:
Saving = True

# Tracks Data Extraction
Tracks = False # on off switch
if Tracks == True:
    print('Tracks:')
    data = {}
    for g in groups:
        data[g] = {}
        for m in Tracksdatalabels:
            data[g][m] = []
    for f in folders:
        excelfiles = os.listdir(f)
        for e in excelfiles:
            if 'Tracks' in e:
                print(e)
                output = maindatasortTracks(f+ '\\' + e, Tracksdatalabels)
                for m in Tracksdatalabels:
                    data[e[0]][m] = data[e[0]][m] + output[m]
    if Saving == True:
        with pd.ExcelWriter(outputloc, mode='a', if_sheet_exists='replace') as writer: 
            for g in groups: 
                df = pd.DataFrame.from_dict(data[g], orient='index')
                df = df.transpose()
                df.to_excel(writer, sheet_name=g)

# Spot Data Extraction
Spots = True
if Spots == True:
    print('Spots:')
    data = pd.DataFrame(columns=['Group', 'Track ID', 'Frame Number', 'Angle'])
    for f in folders:
        excelfiles = os.listdir(f)
        for e in excelfiles:
            if 'Spots' in e:
                print(e)
                output = spotssorting(f+ '\\' + e)
                for id in output.keys():
                    idangles = []
                    for frame in output[id].keys():
                        data.loc[len(data.index)] = [e[0], id, frame, output[id][frame]]
    if Saving == True:
        with pd.ExcelWriter(outputloc, mode='a', if_sheet_exists='replace') as writer: 
            data.to_excel(writer, sheet_name=f'Spots 2.0')

C1_LSpots.csv
C1_RSpots.csv
C2_LSpots.csv
G1_LSpots.csv
G2_LSpots.csv
G2_RSpots.csv
G3_RSpots.csv
T1_LSpots.csv
T1_RSpots.csv
T2_LSpots.csv
T2_RSpots.csv
C1_LSpots.csv
C1_RSpots.csv
C2_LSpots.csv
G1_LSpots.csv
G2_RSpots.csv
G3_LSpots.csv
G3_RSpots.csv
T1_LSpots.csv
T1_RSpots.csv
T2_LSpots.csv
T3_LSpots.csv
T3_RSpots.csv
